# メッシュデータ変換処理

点群データからメッシュデータへの変換，およびメッシュデータから点群データへの変換について述べます．
点群データからメッシュデータへの変換を行うアルゴリズムとしては，マーチングキューブ法が有名です．
まずはじめに，点群データを2値のボクセルデータないし（物体表面上のボクセルが物体表面までの距離に応じた連続値である）TSDFデータに変換します．
Open3Dでは，たとえば下記の処理で（空の）TSDFデータ`volume`を定義します．

In [1]:
import open3d as o3d

volume = o3d.pipelines.integration.ScalableTSDFVolume(
    voxel_length=4.0 / 512.0,
    sdf_trunc=0.04,
    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


次に，複数枚のRGBD画像とそのオドメトリを用いて，`volume`に点群データの情報を統合していきます．
オドメトリの推定方法については`o3d_rgbd_odometry.ipynb`を参照ください．
今回のサンプルプログラムでは，既に取得したオドメトリをファイルから読み込んで使用します．

In [2]:
import numpy as np

class CameraPose:
    def __init__(self, meta, mat):
        self.metadata = meta
        self.pose = mat
    def __str__(self):
        return 'Metadata : ' + ' '.join(map(str, self.metadata)) + "\nPose : \n" + np.array_str(self.pose)

def read_trajectory(filename):
    traj = []
    with open(filename, 'r') as f:
        metastr = f.readline()
        while metastr:
            metadata = list(map(int, metastr.split()))
            mat = np.zeros(shape=(4, 4))
            for i in range(4):
                matstr = f.readline()
                mat[i] = np.fromstring(matstr, dtype=float, sep=' \t')
            traj.append(CameraPose(metadata, mat))
            metastr = f.readline()
    return traj

dirname = "../3rdparty/Open3D/examples/test_data"
camera_poses = read_trajectory(dirname + "/RGBD/odometry.log")

これで，ファイルodometry.logからオドメトリを読み込むことができました．
次に，RGBD画像を次々に読み込み，オドメトリを用いて点群データを幾何変換しつつ，`volume`にデータを格納します．

In [3]:
for i in range(len(camera_poses)):
    print("Integrate {:d}-th image into the volume.".format(i))
    color = o3d.io.read_image(dirname + "/RGBD/color/{:05d}.jpg".format(i))
    depth = o3d.io.read_image(dirname + "/RGBD/depth/{:05d}.png".format(i))
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    volume.integrate(
        rgbd,
        o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault),
        np.linalg.inv(camera_poses[i].pose))

Integrate 0-th image into the volume.
Integrate 1-th image into the volume.
Integrate 2-th image into the volume.
Integrate 3-th image into the volume.
Integrate 4-th image into the volume.


こうして，TSDFデータを取得することができました．

ここで，マーチングキューブ法を用いてTSDFデータをメッシュデータに変換するアルゴリズムを説明します．
マーチングキューブ法は，物体の表面が通るボクセル，すなわち今回の例ではTSDF値が$1$から$-1$までの何らかの連続値を取るようなボクセルに対して，メッシュ（等値面）を切るという方法です．
ボクセルの$8$個の頂点それぞれが物体の内側にあるか外側にあるかを考えると，メッシュの切り方のパターンは全部で$2^8 = 256$通りとなりますが，回転対称や反転を無視すると，全パターンは$15$通りとなります．
（とはいえ，実際にボクセルをメッシュに変換するプログラムの中では回転・反転操作を施したメッシュを区別するため全$256$通りを試すことになります．）
仮にボクセルの頂点が物体の外側にある場合の値を$1$，内側にある場合の値を$0$としたときに，
辺の両端の頂点の持つ値が異なるようなボクセルの辺上にメッシュの頂点が存在することになります．
ここで，TSDF値は各ボクセルと物体表面との距離を表すため，ボクセルの頂点の値は２値ではなく，（メッシュとの近さを表す）連続値として算出することができます．
この値に基づいて線形補間することで，メッシュ頂点の座標を決定します．

では，先程取得したTSDFデータ`volume`からメッシュデータを作成し，描画してみましょう．

In [4]:
print("Extract a triangle mesh from the volume and visualize it.")
mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh],
                                  front=[0.5297, -0.1873, -0.8272],
                                  lookat=[2.0712, 2.0312, 1.7251],
                                  up=[-0.0558, -0.9809, 0.1864],
                                  zoom=0.47)

Extract a triangle mesh from the volume and visualize it.


上記を実行すると，作成されたメッシュデータが描画されます．
マーチングキューブ法によるメッシュデータの作成は上記の2行目で行われています．

最後に，作成されたメッシュデータから点群データをサンプリングし，表示させてみましょう．

In [5]:
print("Extract a point cloud from the mesh and visualize it.")
pcd = mesh.sample_points_uniformly(number_of_points=10000)
o3d.visualization.draw_geometries([pcd],
                                  front=[0.5297, -0.1873, -0.8272],
                                  lookat=[2.0712, 2.0312, 1.7251],
                                  up=[-0.0558, -0.9809, 0.1864],
                                  zoom=0.47)

Extract a point cloud from the mesh and visualize it.


上記のプログラムを実行すると，抽出された点群データが描画されます．
なお，今回の例では`sample_points_uniformly()`という関数を使用しましたが，これは各メッシュ上の点を（メッシュの面積に比例した個数だけ）ランダムサンプリングするという単純なアルゴリズムを用いています．
他にも，たとえばPoisson Disk Samplingというサンプリング手法を使う選択肢もあります．
これを用いることで，空間的均一性の高い点群データを得ることができます．